# Exploratory data analysis  
In this notebook I explored the get_around_delay_analysis data to define the distribution of different variables and identify which variables would aide the most in answering the following questions:  
- What is the best minimun delay between two rentals  (threshold)  
- What is the scope of such new feature: mobile vs connect rentals  
- What is the frequency of drivers being late and the impact on the next driver (rental canceled or not)

## Import libraries

In [33]:
import pandas as pd
import numpy as np
import plotly.express as px
import os 

In [34]:
# Define current dir (needed for when deploying app in Heroku)
curren_dir = os.getcwd()
curren_dir

'/Users/aura.moreno/Documents/Jedha/fullstack_datascience/05-Getaround_app_deployment/dashboard'

## Load data

In [35]:
# Load data
file_name = os.path.join(curren_dir,'data/raw/get_around_delay_analysis.xlsx')

df = pd.read_excel(io = file_name,engine=None,sheet_name='rentals_data')
print("Shape of delay data:",df.shape)
df.head()


Shape of delay data: (21310, 7)


,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000.0,363965.0,mobile,canceled,NaN,NaN,NaN
1,507750.0,269550.0,mobile,ended,-81.0,NaN,NaN
2,508131.0,359049.0,connect,ended,70.0,NaN,NaN
3,508865.0,299063.0,connect,canceled,NaN,NaN,NaN
4,511440.0,313932.0,mobile,ended,NaN,NaN,NaN


In [36]:
# Info on data variables and missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21310 entries, 0 to 21309
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   rental_id                                   21310 non-null  float64
 1   car_id                                      21310 non-null  float64
 2   checkin_type                                21310 non-null  object 
 3   state                                       21310 non-null  object 
 4   delay_at_checkout_in_minutes                16346 non-null  float64
 5   previous_ended_rental_id                    1841 non-null   float64
 6   time_delta_with_previous_rental_in_minutes  1841 non-null   float64
dtypes: float64(5), object(2)
memory usage: 1.1+ MB


In [37]:
# Convert rental_id, car_id and previous_ended_rental_id to string
df['rental_id'] = [str(i) for i in df.rental_id.tolist()]
df['car_id'] = [str(i) for i in df.car_id.tolist()]
df['previous_ended_rental_id'] = [str(i) for i in df.previous_ended_rental_id.tolist()]

In [38]:
# Basic description
df.describe(include='all')

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,21310,21310,21310,21310,16346.000000,21310,1841.000000
unique,21310,8143,2,2,NaN,1789,NaN
top,505000.0,334190.0,mobile,ended,NaN,nan,NaN
freq,1,33,17003,18045,NaN,19469,NaN
mean,NaN,NaN,NaN,NaN,59.701517,NaN,279.288430
std,NaN,NaN,NaN,NaN,1002.561635,NaN,254.594486
min,NaN,NaN,NaN,NaN,-22433.000000,NaN,0.000000
25%,NaN,NaN,NaN,NaN,-36.000000,NaN,60.000000
50%,NaN,NaN,NaN,NaN,9.000000,NaN,180.000000
75%,NaN,NaN,NaN,NaN,67.000000,NaN,540.000000


We see that we have some very extreme values for the delay at checkout in minutes.  
Show this as a plot, and then filter out such extreme values

In order to have straight away the info on whether if the cancelled rental had a previous late check in, create a second dataframe

In [39]:
# Add column to indicate if rental checkin late
df['checkout_late'] = ['late' if delay>0  else 'early' if delay<0 else 'notapplicable' for delay in df.delay_at_checkout_in_minutes]
df.checkout_late.value_counts()

late             9404
early            6820
notapplicable    5086
Name: checkout_late, dtype: int64

In [40]:
# Dataframe giving information for previously ended rental
df_2 = df.copy()
df_2 = df[['rental_id', 'car_id','checkin_type','checkout_late', 'delay_at_checkout_in_minutes']]
df_2 = df_2.rename(columns={'rental_id':'previous_ended_rental_id', 'checkout_late':'previous_ended_rental_late', 'delay_at_checkout_in_minutes':'previous_ended_rental_checkout_delay'})
print(df_2.shape)
df_2.head()

(21310, 5)


,previous_ended_rental_id,car_id,checkin_type,previous_ended_rental_late,previous_ended_rental_checkout_delay
0,505000.0,363965.0,mobile,notapplicable,NaN
1,507750.0,269550.0,mobile,early,-81.0
2,508131.0,359049.0,connect,late,70.0
3,508865.0,299063.0,connect,notapplicable,NaN
4,511440.0,313932.0,mobile,notapplicable,NaN


In [41]:
# Merged dataframes
df_merged = df.merge(df_2,how="left")
print(df_merged.shape)
df_merged.head(20)

(21310, 10)


,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,checkout_late,previous_ended_rental_late,previous_ended_rental_checkout_delay
0,505000.0,363965.0,mobile,canceled,NaN,nan,NaN,notapplicable,NaN,NaN
1,507750.0,269550.0,mobile,ended,-81.0,nan,NaN,early,NaN,NaN
2,508131.0,359049.0,connect,ended,70.0,nan,NaN,late,NaN,NaN
3,508865.0,299063.0,connect,canceled,NaN,nan,NaN,notapplicable,NaN,NaN
4,511440.0,313932.0,mobile,ended,NaN,nan,NaN,notapplicable,NaN,NaN
5,511626.0,398802.0,mobile,ended,-203.0,nan,NaN,early,NaN,NaN
6,511639.0,370585.0,connect,ended,-15.0,563782.0,570.0,early,late,136.0
7,512303.0,371242.0,mobile,ended,-44.0,nan,NaN,early,NaN,NaN
8,512475.0,322502.0,mobile,canceled,NaN,nan,NaN,notapplicable,NaN,NaN
9,513434.0,256528.0,connect,ended,23.0,nan,NaN,late,NaN,NaN


In [42]:
# Add a column indicating  delta between chechekin delay and time delta between chained rentals
df_merged['delta_checkin_previous_rental'] = df_merged.time_delta_with_previous_rental_in_minutes - df_merged.previous_ended_rental_checkout_delay
df_merged['delta_checkin_previous_rental'].describe()

count     1714.000000
mean       300.679697
std        496.907112
min     -12548.000000
25%         48.000000
50%        201.500000
75%        574.000000
max       4684.000000
Name: delta_checkin_previous_rental, dtype: float64

In [43]:
# Boolean column for problematic cases
df_merged['delta_checkin_previous_rental_problematic_bool'] = ['yes' if delay<0 else 'no' for delay in df_merged.delta_checkin_previous_rental.tolist()]

In [44]:
# Add column indicating if rental is being succeceded in less than 12 hours
df_merged['succesive_rental'] = ['yes' if float(rental)>0 else 'no' for rental in df_merged.previous_ended_rental_id.tolist()]
df_merged['succesive_rental'].value_counts()

no     19469
yes     1841
Name: succesive_rental, dtype: int64

In [45]:
df_merged.head(10)

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,checkout_late,previous_ended_rental_late,previous_ended_rental_checkout_delay,delta_checkin_previous_rental,delta_checkin_previous_rental_problematic_bool,succesive_rental
0,505000.0,363965.0,mobile,canceled,NaN,nan,NaN,notapplicable,NaN,NaN,NaN,no,no
1,507750.0,269550.0,mobile,ended,-81.0,nan,NaN,early,NaN,NaN,NaN,no,no
2,508131.0,359049.0,connect,ended,70.0,nan,NaN,late,NaN,NaN,NaN,no,no
3,508865.0,299063.0,connect,canceled,NaN,nan,NaN,notapplicable,NaN,NaN,NaN,no,no
4,511440.0,313932.0,mobile,ended,NaN,nan,NaN,notapplicable,NaN,NaN,NaN,no,no
5,511626.0,398802.0,mobile,ended,-203.0,nan,NaN,early,NaN,NaN,NaN,no,no
6,511639.0,370585.0,connect,ended,-15.0,563782.0,570.0,early,late,136.0,434.0,no,yes
7,512303.0,371242.0,mobile,ended,-44.0,nan,NaN,early,NaN,NaN,NaN,no,no
8,512475.0,322502.0,mobile,canceled,NaN,nan,NaN,notapplicable,NaN,NaN,NaN,no,no
9,513434.0,256528.0,connect,ended,23.0,nan,NaN,late,NaN,NaN,NaN,no,no


## Export dataframe for plotting

In [46]:
file_name = os.path.join(curren_dir,'data/processed/get_around_delay_analysis.csv')
df_merged.to_csv(file_name, index=False)

## Plots

In [66]:
# Total of ended rentals being late
dfg = df_merged.groupby(['checkout_late']).size().reset_index(name='counts')
dfg['proportion_of_total_rentals'] = dfg.counts / sum(dfg.counts)
print(sum(dfg.counts))
display(dfg)

fig = px.bar(dfg, x="checkout_late", y ='proportion_of_total_rentals', 
title=" Fig1. Check-out late status of all rentals (n ={} rentals; {} successive rentals)".format(sum(dfg.counts), df_merged.succesive_rental.value_counts()['yes']))
fig.show(renderer='notebook_connected')

21310


,checkout_late,counts,proportion_of_total_rentals
0,early,6820,0.320038
1,late,9404,0.441295
2,notapplicable,5086,0.238667


In [65]:
# Total of ended rentals being late
dfg = df_merged.groupby(['checkin_type','checkout_late']).size().reset_index(name='counts')
print(sum(dfg.counts))
display(dfg)

tot_rentals_mobile = df_merged.checkin_type.value_counts()['mobile']
tot_rentals_connect = df_merged.checkin_type.value_counts()['connect']

fig = px.histogram(df_merged, x="checkin_type", barmode='group',barnorm='percent', color="checkout_late",color_discrete_sequence=px.colors.qualitative.D3,
title="Fig2.Check-out late status of all rentals (n ={} mobile rentals; n = {} connect rentals)".format(tot_rentals_mobile, tot_rentals_connect))
fig.show(renderer='notebook_connected')

21310


,checkin_type,checkout_late,counts
0,connect,early,1918
1,connect,late,1459
2,connect,notapplicable,930
3,mobile,early,4902
4,mobile,late,7945
5,mobile,notapplicable,4156


In [49]:
# Total of succeded rentals per application
dfg = df_merged.groupby(['checkin_type', 'succesive_rental']).size().reset_index(name='counts')
display(dfg)

suc_rentals_mob = dfg.counts[(dfg.succesive_rental=='yes') & (dfg.checkin_type=="mobile")].values[0]
suc_rentals_conn = dfg.counts[ (dfg.succesive_rental=='yes') & (dfg.checkin_type=="connect")].values[0]

fig = px.histogram(df_merged, x="checkin_type", barmode='group',barnorm='percent', color="succesive_rental",color_discrete_sequence=px.colors.qualitative.Safe,
title= f'Fig3.Successive rentals proportion (n= {suc_rentals_mob} and {suc_rentals_conn} successive rentals for mobile and connect respectively)')
fig.show(renderer='notebook_connected')

,checkin_type,succesive_rental,counts
0,connect,no,3494
1,connect,yes,813
2,mobile,no,15975
3,mobile,yes,1028


In [50]:
# Distribution of delay at checkout

delay_quantile = df_merged['delay_at_checkout_in_minutes'].quantile((0,0.25,0.5,0.75,1))
print(delay_quantile)

print('Fifty percent of rentals have a delay checkin time between {} and {} minutes'.format(
    delay_quantile.loc[0.25],delay_quantile.loc[0.75] )
    )


fig = px.histogram(df_merged, x="delay_at_checkout_in_minutes", nbins=200, marginal="box", title='Fig4.Distribution of delay at checkout',
labels={'delay_at_checkout_in_minutes': 'Delay at chekout (mins)'

})
fig.show(renderer='notebook_connected')

0.00   -22433.0
0.25      -36.0
0.50        9.0
0.75       67.0
1.00    71084.0
Name: delay_at_checkout_in_minutes, dtype: float64
Fifty percent of rentals have a delay checkin time between -36.0 and 67.0 minutes


In [51]:
# Count how many checkouts have a delay that is more than 12 hours (meaning that despite what is said in the documentation, some delays where computed)
mask_12hrs = df_merged['delay_at_checkout_in_minutes'] >= (12*60)
delays_12hrs = sum(mask_12hrs)
print(f'There are {delays_12hrs} cases where there is a late checkout delay of more than 12 hours')

There are 494 cases where there is a late checkout delay of more than 12 hours


In [52]:
# Filter for shorter delays
print('Data shape before filter:',df_merged.shape)
df_merged_clean = df_merged.copy()
df_merged_clean = df_merged[~mask_12hrs]
print('Data shape after filter:',df_merged.shape)


Data shape before filter: (21310, 13)
Data shape after filter: (21310, 13)


In [53]:
# Distribution of delay at checkout: Focus on delays of  4 hours
df_merged_filt = df_merged.copy()
delay_hours = 4
filter_mask = abs(df_merged_filt['delay_at_checkout_in_minutes']) <= delay_hours*60
df_merged_filt = df_merged_filt[filter_mask]

fig = px.histogram(df_merged_filt, x="delay_at_checkout_in_minutes", marginal="box",
nbins=200, title=f'Fig5.Distribution of delay at checkout (max {delay_hours}hrs absolute)', 
labels={'delay_at_checkout_in_minutes': 'Delay at chekout (mins)'

})


fig.show(renderer='notebook_connected')


In [54]:
# Distribution for late checkouts only
df_merged_filt = df_merged.copy()
delay_hours = 4
filter_mask = abs(df_merged_filt['delay_at_checkout_in_minutes']) <= delay_hours*60
filter_mask2 = df_merged_filt['checkout_late'] == 'late'
df_merged_filt = df_merged_filt[filter_mask & filter_mask2]

fig = px.histogram(df_merged_filt, x="delay_at_checkout_in_minutes", marginal="box",
nbins=200, title=f'Fig6.Distribution of delay at checkout when late (max {delay_hours}hrs late)', 
labels={'delay_at_checkout_in_minutes': 'Delay at chekout (mins)'

})


fig.show(renderer='notebook_connected')

In [55]:
# Frequency of being late, and impact on next driver: take a look at if previous rental was late, and whether the state of actual rental

dfg = df_merged.groupby(['previous_ended_rental_late', 'state']).size().reset_index(name='counts')
print(sum(dfg.counts))
display(dfg)


fig = px.histogram(df_merged, x="previous_ended_rental_late", barmode='group',barnorm='percent', color="state",
title="Fig7.Cancelled and ended status for different delay categories")
fig.show(renderer='notebook_connected')

1824


,previous_ended_rental_late,state,counts
0,early,canceled,100
1,early,ended,734
2,late,canceled,105
3,late,ended,759
4,notapplicable,canceled,22
5,notapplicable,ended,104


In [56]:
# Separate by mobile app
dfg = df_merged.groupby(['previous_ended_rental_late', 'checkin_type', 'state']).size().reset_index(name='counts')
print(sum(dfg.counts))
display(dfg)


fig = px.histogram(df_merged, x="previous_ended_rental_late", facet_col="checkin_type",barmode='group', color="state",
title="Fig8.Cancelled and ended status for different delay categories")
fig.show(renderer='notebook_connected')

1824


,previous_ended_rental_late,checkin_type,state,counts
0,early,connect,canceled,66
1,early,connect,ended,409
2,early,mobile,canceled,34
3,early,mobile,ended,325
4,late,connect,canceled,55
5,late,connect,ended,242
6,late,mobile,canceled,50
7,late,mobile,ended,517
8,notapplicable,connect,canceled,9
9,notapplicable,connect,ended,20


In [57]:
# Show proportions
fig = px.histogram(df_merged, x="previous_ended_rental_late", facet_col="checkin_type",barmode='group',barnorm='percent', color="state",
title="Fig9.Cancelled and ended status for different delay categories: proportions")
fig.show(renderer='notebook_connected')

#### Focus on successive rentals

In [58]:
# Proportion of problematic delays with respect to time beween chekcout and checkin and cancelled status

df_merged_filt = df_merged.copy()
filter_mask = df_merged.succesive_rental=="yes"
df_merged_filt = df_merged_filt[filter_mask]

dfg = df_merged_filt.groupby(['delta_checkin_previous_rental_problematic_bool']).size().reset_index(name='counts')
print(sum(dfg.counts))
display(dfg)


fig = px.histogram(df_merged_filt, x="checkin_type", barmode='group', color="delta_checkin_previous_rental_problematic_bool",
title="Fig10.Impact of late check-in on next rental", 
labels={'delta_checkin_previous_rental_problematic_bool': 'problematic late checkin',
'checkin_type': 'Check-in type'})
fig.show(renderer='notebook_connected')

1841


,delta_checkin_previous_rental_problematic_bool,counts
0,no,1623
1,yes,218


In [59]:
# Effect of delay with previous rental: compare dist of delay and color canceled ended mobile vs connect for successive rentals only
# Plot for absolute deltas of max 4 hours (240 mins)
df_merged_filt = df_merged.copy()
delay_hours = 2
filter_mask = abs(df_merged_filt['delta_checkin_previous_rental']) <= delay_hours*60
filter_mask2 = df_merged_filt.succesive_rental=="yes"
df_merged_filt = df_merged_filt[filter_mask & filter_mask2]


fig = px.box(df_merged_filt, x="state", y="delta_checkin_previous_rental", 
boxmode='group', points='all', facet_col ="checkin_type", 
                labels={
                    "parameter": "Rental status",
                    "delta_checkin_previous_rental": "Delta between checkout and new checkin (mins)",
                    "delta_checkin_previous_rental_problematic": "Time delta category"
                },
                title='Fig11.Delta of time between succesive rental and delay of checkout (max 4 hours displayed)')

fig.show(renderer='notebook_connected')

In [60]:
# Categorize delta of previous rental checkin delay and time between next rental. Use bins
bins = [-np.inf,0, 10, 30, 60, np.inf]
labs = ['problematic' ,'[0-10 mins]', '[10-30 mins]', '[30-60 mins]', '[>60mins]']
df_merged['delta_checkin_previous_rental_problematic'] = pd.cut(df_merged['delta_checkin_previous_rental'], bins = bins, labels=labs)


dfg = df_merged.groupby(['state','delta_checkin_previous_rental_problematic']).size().reset_index(name='counts')
display(dfg)

df_merged_filt = df_merged.copy()
filter_mask = df_merged_filt.succesive_rental=="yes"
df_merged_filt = df_merged_filt[filter_mask]
df_merged_filt.dropna(subset=['delta_checkin_previous_rental_problematic'], inplace=True)

fig = px.histogram(df_merged_filt, x="state", barmode='group',barnorm='percent', color="delta_checkin_previous_rental_problematic",color_discrete_sequence=px.colors.qualitative.D3,
title="Fig12.Category enrichment for time delta of checkout and new checkin for successive rentals", 
labels={
   'delta_checkin_previous_rental_problematic': 'Delta checkout vs new checkin category' 
})
fig.show(renderer='notebook_connected')

,state,delta_checkin_previous_rental_problematic,counts
0,canceled,problematic,37
1,canceled,[0-10 mins],1
2,canceled,[10-30 mins],9
3,canceled,[30-60 mins],4
4,canceled,[>60mins],154
5,ended,problematic,186
6,ended,[0-10 mins],41
7,ended,[10-30 mins],89
8,ended,[30-60 mins],112
9,ended,[>60mins],1081


In [61]:
# Impact of threshold on number of problematic cases and impact on ended rentals not being able to take place

# Prepare new colums
threshold = 45

df_merged['new_delta_checkin_previous_rental'] = threshold - df_merged.previous_ended_rental_checkout_delay
df_merged['new_thresh_problematic_case'] = ['no' if delay<0 else 'yes' for delay in df_merged.new_delta_checkin_previous_rental.tolist()]

# Cases which also become problematic: those that have a lower time delta between checkin and chekout than new threshold
new_thres_problematic = (df_merged['time_delta_with_previous_rental_in_minutes'] < threshold)
df_merged.loc[new_thres_problematic, 'new_thresh_problematic_case'] = 'yes'

    

In [62]:
# Count number of problematic cases before and after new threshold
# A new problematic case will be defined in 
df_merged_filt = df_merged.copy()
filter_mask = df_merged_filt.succesive_rental=="yes"
df_merged_filt = df_merged_filt[filter_mask]


dfg = df_merged_filt.delta_checkin_previous_rental_problematic_bool.value_counts()
display(dfg)

problematic_cases = df_merged_filt.delta_checkin_previous_rental_problematic_bool.value_counts()['yes']

dfg_new = df_merged_filt.groupby(['delta_checkin_previous_rental_problematic_bool','new_thresh_problematic_case' ]).size().reset_index(name='counts')
display(dfg_new)

prev_problematic_cases_solved = dfg_new.loc[(dfg_new.delta_checkin_previous_rental_problematic_bool=='yes') & (dfg_new.new_thresh_problematic_case=="no"), 'counts'].tolist()[0]

impacted_ended_rentals_now_problematic = dfg_new.loc[(dfg_new.delta_checkin_previous_rental_problematic_bool=='no') & (dfg_new.new_thresh_problematic_case=="yes"), 'counts'].tolist()[0]


print(sum(dfg_new.counts))
print(f'After using a threshold of {threshold} minutes, there were {prev_problematic_cases_solved} previous problematic cases solved. However, previously ended rentals which would no longer take place with new threshold are: {impacted_ended_rentals_now_problematic}')



no     1623
yes     218
Name: delta_checkin_previous_rental_problematic_bool, dtype: int64

,delta_checkin_previous_rental_problematic_bool,new_thresh_problematic_case,counts
0,no,no,282
1,no,yes,1341
2,yes,no,72
3,yes,yes,146


1841
After using a threshold of 45 minutes, there were 72 previous problematic cases solved. However, previously ended rentals which would no longer take place with new threshold are: 1341


In [63]:
dfg_new_scope = df_merged_filt.groupby(['checkin_type', 'delta_checkin_previous_rental_problematic_bool','new_thresh_problematic_case' ]).size().reset_index(name='counts')
display(dfg_new_scope)

mask_connect = (dfg_new_scope.checkin_type=="connect") & (dfg_new_scope.delta_checkin_previous_rental_problematic_bool=='yes') & (dfg_new_scope.new_thresh_problematic_case=="no")
mask_mobile =  (dfg_new_scope.checkin_type=="mobile") & (dfg_new_scope.delta_checkin_previous_rental_problematic_bool=='yes') & (dfg_new_scope.new_thresh_problematic_case=="no")

mask_connect_2 = (dfg_new_scope.checkin_type=="connect") & (dfg_new_scope.delta_checkin_previous_rental_problematic_bool=='no') & (dfg_new_scope.new_thresh_problematic_case=="yes")
mask_mobile_2 = (dfg_new_scope.checkin_type=="mobile") & (dfg_new_scope.delta_checkin_previous_rental_problematic_bool=='no') & (dfg_new_scope.new_thresh_problematic_case=="yes")


prev_problematic_cases_solved_connect = dfg_new_scope.loc[mask_connect, 'counts'].tolist()[0]
prev_problematic_cases_solved_mobile = dfg_new_scope.loc[mask_mobile, 'counts'].tolist()[0]


impacted_ended_rentals_now_problematic_conn = dfg_new_scope.loc[mask_connect_2, 'counts'].tolist()[0]
impacted_ended_rentals_now_problematic_mob = dfg_new_scope.loc[mask_mobile_2, 'counts'].tolist()[0]

print(f'''After using a threshold of {threshold} minutes, there were {prev_problematic_cases_solved} previous problematic cases solved ({prev_problematic_cases_solved_connect} mobile and {prev_problematic_cases_solved_mobile} connect).
However, previously ended rentals which would no longer take place with new threshold are: {impacted_ended_rentals_now_problematic} ({impacted_ended_rentals_now_problematic_conn}  mobile and {impacted_ended_rentals_now_problematic_mob} connect)''')



,checkin_type,delta_checkin_previous_rental_problematic_bool,new_thresh_problematic_case,counts
0,connect,no,no,88
1,connect,no,yes,656
2,connect,yes,no,21
3,connect,yes,yes,48
4,mobile,no,no,194
5,mobile,no,yes,685
6,mobile,yes,no,51
7,mobile,yes,yes,98


After using a threshold of 45 minutes, there were 72 previous problematic cases solved (21 mobile and 51 connect).
However, previously ended rentals which would no longer take place with new threshold are: 1341 (656  mobile and 685 connect)


In [64]:
# Plot a Sankey diagram to better see evolution of cases
# Use documentation to produce plot: https://plotly.com/python/sankey-diagram/#basic-sankey-diagram

import plotly.graph_objects as go


fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = ["not problematic", "problematic", "not problematic", "problematic"],
      color = ["darkblue", 'darkred', "blue", "red"]
    ),
    link = dict(
      source = [0, 0, 1, 1], # indices correspond to labels
      target = [2, 3, 2, 3],
      value = dfg_new.counts.tolist()
  ))])

fig.update_layout(title_text=f"Fig13.Evolution of number of problematic cases due to late check in after new threshold: {threshold} mins", font_size=10)
fig.show(renderer='notebook_connected')